In [ ]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained("openai-community/gpt2", add_prefix_space=True)
tokenizer("Hello world")["input_ids"]

In [ ]:
tokenizer("Hello world")

In [ ]:
tokenizer("Hello worldly beings")

In [ ]:
tokenizer("Hello otherworldly beings")

In [ ]:
type(tokenizer)

In [ ]:
tokenizer.decode([15496, 995])

In [ ]:
for text in ('Hello world', 'Hello worldly beings', 'Hello otherworldly beings'):
    print(tokenizer.decode(tokenizer(text)['input_ids']))

In [ ]:
for text in ('Hello world', 'Hello worldly beings', 'Hello otherworldly beings'):
    print(tokenizer.encode(text))

In [ ]:
for text in ('Hello world', 'Hello worldly beings', 'Hello otherworldly beings'):
    print([tokenizer.decode(id) for id in tokenizer(text, is_split_into_words=True)['input_ids']])

In [ ]:
for text in ('Hello world', 'Hello worldly beings', 'Hello otherworldly beings'):
    print(tokenizer.convert_ids_to_tokens(tokenizer(text)['input_ids']))

In [ ]:
tokenizer.get_added_vocab()

In [ ]:
for text in ('Hello world', 'Hello worldly beings', 'Hello otherworldly beings'):
    print(tokenizer.tokenize(text))

In [ ]:
def clean_tokenized_text(tokenized_text):
    words = [wd.replace('Ġ', ' ') if wd.startswith('Ġ') else '#' + wd for wd in tokenized_text]
    return ''.join(words)

In [ ]:
with open('28_sample_en_text.txt') as f:
    text = f.read()

print(clean_tokenized_text(tokenizer.tokenize(text)))

It's clear that GPT was trained with a high number of merges, because there are barely any words that get split. Still, there are some; here's an example:

In [ ]:
tokenizer.tokenize('debutant')

I bet that if I train a BPE encoder with the bible with a low number of merges, there will be many more splits. The question is how much time it would take to train with the maximum number of merges.